In [2]:
import numpy as np

from lightfm.datasets import fetch_movielens

movielens = fetch_movielens()

/Users/arthursaprykin/anaconda3/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [3]:
for key, value in movielens.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse.csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


In [4]:
train = movielens['train']
test = movielens['test']

In [5]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM()
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.45, test 0.13.


In [6]:
model = LightFM(learning_rate=0.05, loss='warp')

model.fit_partial(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.60, test 0.22.


In [7]:
def sample_recommendation(model, data, user_ids):
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()                                    
                          [user_id].indices]
        
        scores = model.predict(user_id, np.arange(n_items))

        top_items = data['item_labels'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("     Known positives:")
        
        for x in known_positives[:3]:
            print("        %s" % x)
        
        print("     Recommended:")
        
        for x in top_items[:3]:
            print("        %s" % x)

In [18]:
movielens['train'].tocsr()

<943x1682 sparse matrix of type '<class 'numpy.float32'>'
	with 90570 stored elements in Compressed Sparse Row format>

In [8]:
movielens['train'].tocsr()[10].indices

array([  7,   8,  10,  11,  14,  21,  23,  24,  27,  28,  38,  39,  41,
        46,  50,  51,  53,  55,  56,  57,  68,  69,  78,  82,  85,  87,
        89,  93,  96,  97,  99, 106, 108, 119, 120, 122, 124, 134, 167,
       172, 174, 175, 179, 184, 189, 190, 193, 195, 202, 203, 207, 210,
       212, 214, 215, 221, 227, 228, 229, 236, 237, 238, 240, 257, 258,
       259, 267, 273, 276, 285, 289, 290, 299, 300, 311, 316, 317, 323,
       331, 349, 355, 356, 364, 366, 369, 371, 381, 382, 385, 392, 394,
       398, 400, 401, 404, 413, 422, 426, 427, 428, 429, 430, 432, 433,
       434, 448, 450, 454, 503, 507, 516, 520, 523, 525, 526, 543, 548,
       560, 572, 576, 579, 596, 602, 645, 651, 653, 658, 659, 661, 662,
       689, 691, 698, 706, 709, 712, 713, 714, 715, 716, 717, 718, 719,
       720, 721, 723, 725, 726, 727, 728, 729, 730, 732, 733, 734, 735,
       736, 737, 738, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749,
       750, 751], dtype=int32)

In [9]:
movielens['item_labels']

array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
       'Sliding Doors (1998)', 'You So Crazy (1994)',
       'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object)

In [10]:
sample_recommendation(model, movielens, [10, 25, 451])

User 10
     Known positives:
        Babe (1995)
        Dead Man Walking (1995)
        Seven (Se7en) (1995)
     Recommended:
        Raiders of the Lost Ark (1981)
        Star Wars (1977)
        Return of the Jedi (1983)
User 25
     Known positives:
        Toy Story (1995)
        Twelve Monkeys (1995)
        Dead Man Walking (1995)
     Recommended:
        Contact (1997)
        Scream (1996)
        Liar Liar (1997)
User 451
     Known positives:
        Twelve Monkeys (1995)
        Babe (1995)
        Postino, Il (1994)
     Recommended:
        Raiders of the Lost Ark (1981)
        2001: A Space Odyssey (1968)
        Casablanca (1942)
